In [2]:
# utilities
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import geopandas as gpd
import getpass
import ast
import numpy as np
import time

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import requests

# geo and rester imports
from shapely.geometry import Polygon, MultiPolygon, box
from shapely.ops import transform as shapely_transform
from shapely.wkt import loads
from pyproj import Geod
from functools import partial

from pyproj import Transformer

from rasterio.io import MemoryFile
import rasterio.merge
import rasterio
import rasterio.plot
from rasterio.plot import show
from rasterio.mask import mask

from rasterio.transform import array_bounds, from_bounds, xy

# sentinel imports
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

# DL imports
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm

/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-27 12:07:13.234732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740658033.865009    3512 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740658034.047410    3512 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 12:07:15.917697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-cri

# Get Geo data

In [4]:
# get gdf data
all_gdf = pd.read_csv(r'/home/azureuser/cloudfiles/code/Users/richesmith/Eurosat Training/all_gdf.csv')

In [7]:
all_gdf.head(50)

,geo_labelw,geo_label,geo_code,geometry,new_geometry,aoi_coords_wgs84,aoi_size
0,NaN,South Ayrshire,S12000028,MULTIPOLYGON (((244282.59999999963 632703.5999...,POLYGON ((244282.59999999963 632703.5999999996...,"[-5.0606574611546895, 54.997783671505985, -4.3...","(4368, 6617)"
1,NaN,Huntingdonshire,E07000011,"MULTIPOLYGON (((511946.215 290765.599, 511942....","POLYGON ((511946.215 290765.599, 511942.906 29...","[-0.49990712939176263, 52.15872374092071, 0.05...","(3570, 4865)"
2,NaN,Tewkesbury,E07000083,"POLYGON ((378940.315 221169.788, 378941.813 22...","POLYGON ((378940.315 221169.788, 378941.813 22...","[-2.352748969913588, 51.81930310549422, -1.801...","(3756, 2620)"
3,NaN,West Lancashire,E07000127,"MULTIPOLYGON (((332523.5 410159.094, 332517.72...","POLYGON ((332523.5 410159.094, 332517.724 4101...","[-3.046701000153703, 53.48276587692606, -2.689...","(2359, 2798)"
4,NaN,Darlington,E06000005,"POLYGON ((423801.813 514513.5, 423780.596 5145...","POLYGON ((423801.813 514513.5, 423780.596 5145...","[-1.7097551016536965, 54.45112626483218, -1.40...","(1918, 1912)"
5,NaN,Cheltenham,E07000078,"POLYGON ((391053.906 223329.828, 391055.063 22...","POLYGON ((391053.906 223329.828, 391055.063 22...","[-2.14303258843585, 51.85831272730343, -2.0102...","(902, 907)"
6,NaN,Cambridge,E07000008,"POLYGON ((541592.313 259454.125, 541577.67 259...","POLYGON ((541592.313 259454.125, 541577.67 259...","[0.06861046941714362, 52.157929617258525, 0.18...","(827, 850)"
7,NaN,Erewash,E07000036,"POLYGON ((440746.033 342754.507, 440747.173 34...","POLYGON ((440746.033 342754.507, 440747.173 34...","[-1.4798362085050158, 52.87233913677475, -1.23...","(1589, 1497)"
8,NaN,West Berkshire,E06000037,"POLYGON ((433821.407 164392.405, 433819.188 16...","POLYGON ((433821.407 164392.405, 433819.188 16...","[-1.588087906788207, 51.328963008388456, -0.98...","(4152, 2709)"
9,NaN,Teignbridge,E07000045,"POLYGON ((289094.957 72637.799, 289091.2 72636...","POLYGON ((289094.957 72637.799, 289091.2 72636...","[-3.8828036092716975, 50.46114393798438, -3.42...","(3272, 3339)"


In [8]:
# map geometrys back to Polygon object as its saved as str

def str_to_polygon(polygon_str):
    if not polygon_str.endswith('))'):
        polygon_str += '))'
    return loads(polygon_str)
    
all_gdf['new_geometry'] = all_gdf['new_geometry'].apply(str_to_polygon)

# Helper functions

In [9]:
# splitting the bounding box
# the function splits the bounding box up if it exceeds the max pixels the API will allow (2500) either in width and/or height
# returns as list of boudning boxes and the size for using when stitching back together

# used a few places for help with this...
# https://sentinelhub-py.readthedocs.io/en/latest/examples/large_area_utilities.html
# https://pyproj4.github.io/pyproj/stable/api/geod.html
# 

def split_bounding_box(coords, max_pixels, resolution=10):

    minx, miny, maxx, maxy = coords # gets each piece from the bounding box coords

    # calculates the bounding box width and height in meters as it is in degrees
    # converts degrees to meters
    geod = Geod(ellps="WGS84")
    width = geod.inv(minx, miny, maxx, miny)[2]  # distance between minx and maxx at miny
    height = geod.inv(minx, miny, minx, maxy)[2]  # distance between miny and maxy at minx

    # calculate width/height in pixels
    width_pixels = width / resolution
    height_pixels = height / resolution

    # split into tiles if dimensions exceed max_pixels (2500)
    x_splits = int(np.ceil(width_pixels / max_pixels))
    y_splits = int(np.ceil(height_pixels / max_pixels))

    x_step = (maxx - minx) / x_splits
    y_step = (maxy - miny) / y_splits

    tiles = [] #list to hold tiles
    for i in range(x_splits):
        for j in range(y_splits):
            tile_minx = minx + i * x_step
            tile_miny = miny + j * y_step
            tile_maxx = tile_minx + x_step
            tile_maxy = tile_miny + y_step

            # calculates pixel size for the tile
            tile_width_pixels = int(width_pixels / x_splits)
            tile_height_pixels = int(height_pixels / y_splits)

            # appends bounding box and pixel size to list
            tiles.append(([tile_minx, tile_miny, tile_maxx, tile_maxy], (tile_width_pixels, tile_height_pixels)))

    return tiles # retunrs the list of tiles

In [10]:
# functions to get a token

# help from here...
# https://docs.authlib.org/en/latest/client/oauth2.html

def get_token(client_id, client_secret):
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(
        token_url='https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token',
        client_secret=client_secret,
        include_client_id=True
    )
    token['expires_at'] = time.time() + token['expires_in']  # adds expiry timestamp
    return token

In [ ]:
# token management helper function
# checks time and gets new token when near time expiry
def ensure_valid_token(token, client_id, client_secret):
    if time.time() >= token['expires_at'] - 60:  # Refresh if < 1 min remaining
        print("Token expired. Fetching a new one...")
        token = get_token(client_id, client_secret)
    return token

In [11]:
# credentials
client_id = '2589f2ec-6249-4d95-867c-450a7d83be80'
client_secret = 'clgiyt8zfqPrHAoLSLSKry1wNU26WQGd'

In [13]:
############

# request each tile
# gets an image from sentinel hub and returns as numpy array, 
# then converts numpy array to rasterio memoryfile,
# gets correct geospatial data

# https://rasterio.readthedocs.io/en/stable/api/rasterio.transform.html
# https://medium.com/@mommermiscience/dealing-with-geospatial-raster-data-in-python-with-rasterio-775e5ba0c9f5
# https://docs.sentinel-hub.com/api/latest/evalscript/v3/

#https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/cloudless_mosaic/


def request_tile_rasterio(tile_coords, tile_size, access_token, year):

    # adds timeframe for requestion images
    time_from = f"{year}-05-01T00:00:00Z"
    time_to   = f"{year}-07-30T23:59:59Z"
    mosaicking_order = "leastCC"
    max_cc = 70

    # API request body
    width, height = tile_size
    request_payload = {
        "input": {
            "bounds": {
                "bbox": tile_coords,
                "properties": {
                    # defines CRS as WGS84
                    "crs": "http://www.opengis.net/def/crs/EPSG/0/4326"
                }
            },
            "data": [
                {
                    "type": "sentinel-2-l1c", # uses Setninel 2c 
                    "dataFilter": {
                        "timeRange": {
                            "from": time_from,
                            "to": time_to
                        },
                        "mosaickingOrder": mosaicking_order,
                        "maxCloudCoverage": max_cc
                    }
                }
            ]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [
                {
                    "identifier": "default",
                    "format": {"type": "image/tiff"}
                }
            ]
        },
        "evalscript": """
        //VERSION=3
        function setup() {
          return {
            input: [ "B02", "B03", "B04" ],
            output: { bands: 3 }
          };
        }
        function evaluatePixel(sample) {
          // Return B04=R, B03=G, B02=B
          return [sample.B04, sample.B03, sample.B02];
        }
        """
    }

    # headers
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "image/tiff"
    }

    # make API call
    response = requests.post(
        "https://services.sentinel-hub.com/api/v1/process",
        json=request_payload,
        headers=headers
    )

    # check for errors
    if response.status_code != 200:
        raise RuntimeError(
            f"Request failed with status {response.status_code}: {response.text}"
        )

    # response is a TIFF in bytes
    raw_tiff = response.content

    # this converts to a rater memory file
    memfile = MemoryFile(raw_tiff)
    
    transform = from_bounds(
        tile_coords[0], tile_coords[1], tile_coords[2], tile_coords[3],
        width, height
    )

    # return the memory file:
    return memfile

In [14]:
# stitching tiles
#uses merge function ro merge the memeoryfile objects into a single image - it retains the geospatial positioning
# https://rasterio.readthedocs.io/en/stable/api/rasterio.merge.html

def stitch_tiles_rasterio(tile_memfiles):

    datasets = [rasterio.open(memfile) for memfile in tile_memfiles]
    mosaic, transform = rasterio.merge.merge(datasets)

    # Clean up memory
    for dataset in datasets:
        dataset.close()

    return mosaic, transform

In [15]:
# divides the image into 64x64 pixel sections (in line with Eurosat)
# it excludes patches that have too many invalid pixels - these are out of range of the masked section

# https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html
# https://numpy.org/doc/2.2/reference/maskedarray.html

def split_into_sections(masked_image, mask, section_size, stride=None, threshold=0.2):

    stride = section_size if stride is None else stride
    sections = []
    coordinates = []
    
    channels, height, width = masked_image.shape
    
    for y in range(0, height - section_size + 1, stride):
        for x in range(0, width - section_size + 1, stride):
            section = masked_image[:, y:y + section_size, x:x + section_size]
            mask_section = mask[y:y + section_size, x:x + section_size]  # Extract mask for the section
            
            # computes the fraction of valid pixels (non masked)
            valid_ratio = np.count_nonzero(mask_section) / (section_size * section_size)
            
            if valid_ratio >= threshold:  # only keeps valid pixels exceed threshold
                sections.append(section)
                coordinates.append((x, y))  # stores in top left
            
    return sections, coordinates

In [16]:
#define Vit Model - needed to load model for predictions
# https://huggingface.co/docs/transformers/en/model_doc/vit

class ViTModel(nn.Module):
    def __init__(self, num_classes):
        super(ViTModel, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
        self.model.head = nn.Identity()  # remove the existing head
        self.classifier = nn.Linear(768, num_classes)  # adjusts the feature size based on ViT model used

    def forward(self, x):
        x = self.model(x)
        x = self.classifier(x)
        return x

# load the trained vit model
model_path = '/home/azureuser/cloudfiles/code/Users/richesmith/Eurosat Training/Users/richesmith/Eurosat Training/Models/vit-base-patch16-224-20-epoch_model.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialise the model
num_classes = 10  # 10 classes in the data
model = ViTModel(num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

/tmp/ipykernel_3512/2457408905.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


ViTModel(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
   

In [17]:
# define dataframe to hold data

template_df = pd.DataFrame(columns=['local_authority',
                                      'year',
                                      'total_pixels',
                                      'total_sqkm',
                                      0,
                                      1,
                                      2,
                                      3,
                                      4,
                                      5,
                                      6,
                                      7,
                                      8,
                                      9])

# make predictions and return dataframe

In [18]:
# this pieces everything together, to get images from the Sentinel API, stitch them together where they are too large,
# process them, split the image into 64x64 px sections then make predictions using the Vit model
# the results for each predction are then put into a dataframe aganinst the relevent local authority area and then output to an

# it started off looping through the year list, however it took so long
# to run it sometimes timed out so I ended up doing one year at a time

#year_list = ['2015','2016','2017','2018','2019','2020','2021','2022']

year_list = ['2022']

def make_predictions(df,model,template_df):
    progress_count = 0
    prediction_df = template_df.copy()

    # run function
    token = get_token(client_id, client_secret)
    # loop through local authority labels
    for x in df['geo_label'].tolist():
        filtered_gdf = df[df['geo_label'] == x] # filters to authority
    
        # gets coord of filtered line
        aoi_coord = filtered_gdf.iloc[0].aoi_coords_wgs84 # gets polygon
        
        # currently in a str this covnerts to list
        aoi_coord = ast.literal_eval(aoi_coord) # converts string to list
        
        # gets polygon coord to use later (when overlaying polygon onto boundary image)
        polygon_coord = filtered_gdf.iloc[0].new_geometry

        # then loop through
        for y in year_list:

            try:
                # combining helper functions for processing bounding boxes
                # splits bounding box, requests tiles, stitches together (retains geospatial data)
                # https://sentinelhub-py.readthedocs.io/en/latest/examples/large_area_utilities.html
                def process_large_bounding_box_rasterio(coords, max_pixels=2500, resolution=10):
                    tiles_coords_and_sizes = split_bounding_box(coords, max_pixels, resolution)
                
                    # request all tiles
                    tile_memfiles = [
                        request_tile_rasterio(tile[0], tile[1], token['access_token'],y) for tile in tiles_coords_and_sizes
                    ]
                            
                    # stitch tiles together
                    stitched_image, transform = stitch_tiles_rasterio(tile_memfiles)
                
                    return stitched_image, transform


                # request images for each tile
                tiles_coords_and_sizes = split_bounding_box(aoi_coord, max_pixels=2500, resolution=10)

                # check if token has expired
                token = ensure_valid_token(token, client_id, client_secret)

                # runs function to get image data
                stitched_image, transform = process_large_bounding_box_rasterio(aoi_coord, max_pixels=2500, resolution=10)

                ### enhance image

                # the images coming through from the sentinel hub where very dark, this enhances them so they match the Eurosat images

                # https://scikit-image.org/docs/0.9.x/auto_examples/plot_equalize.html
                # https://medium.com/data-caffeine/enhance-your-image-in-three-ways-in-python-7246a7b6bd66
                #https://stackoverflow.com/questions/60449340/contrast-enhancement-using-a-percentage-cumulative-count-in-matplolib


                # low_percent=2, high_percent=98 gave quite high contrast
                # 5,98 no different really
                def percentile_stretch(image_array, low_percent=2, high_percent=98):
                    # this reshapes to the correct channels so to compute percentiles
                    if image_array.ndim == 3 and image_array.shape[0] < 10:
                        # likely returned shape is (C, H, W)
                        # transpose to (H, W, C)
                        image_reshaped = np.transpose(image_array, (1, 2, 0))
                        channel_first = True
                    else:
                        image_reshaped = image_array
                        channel_first = False

                    # flattens each channel separately
                    c = image_reshaped.shape[2] if image_reshaped.ndim == 3 else 1
                    
                    # hold results in the same shape
                    stretched_image = np.zeros_like(image_reshaped)

                    for ch in range(c):
                        # extract channel
                        channel_data = image_reshaped[..., ch].flatten()
                        
                        # compute percentiles
                        p_low_val = np.percentile(channel_data, low_percent)
                        p_high_val = np.percentile(channel_data, high_percent)

                        # stretch channel, clamp and scale
                        out_channel = image_reshaped[..., ch]
                        out_channel = np.clip(out_channel, p_low_val, p_high_val)
                        out_channel = (out_channel - p_low_val) / (p_high_val - p_low_val + 1e-8)
                        
                        stretched_image[..., ch] = out_channel

                    # if channel_first, transpose back
                    if channel_first:
                        stretched_image = np.transpose(stretched_image, (2, 0, 1))

                    return stretched_image

                # use function and apply
                float_image = stitched_image / 255.0
                enhanced_image = percentile_stretch(float_image, low_percent=0.2, high_percent=99.8)

                ### enhance image end

                # creates polygon from polygon_coord variable
                polygon = Polygon(polygon_coord)
                
                # defines the crs being used 
                raster_crs = "EPSG:4326"  # raster is in WGS84
                polygon_crs = "EPSG:27700"  # the crs the polygon is currently in

                # computes rater bounds gives geographic extent of the raster in [minx, miny, maxx, maxy]
                raster_bounds = array_bounds(
                    stitched_image.shape[1],  # height
                    stitched_image.shape[2],  # width
                    transform
                )
                
                # recomputes the affine transformation using the rasters actual bounds
                # previous transform incoorect. COuld be because the images are being stitched together
                # https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html
                width, height = stitched_image.shape[2], stitched_image.shape[1]
                correct_transform = from_bounds(*raster_bounds, width=width, height=height)
                
                # creates a transformer object to convert from polygon CRS to raster CRS
                transformer = Transformer.from_crs(polygon_crs, raster_crs, always_xy=True)
                
                # reprojects the polygon into the correct crs using shapely
                reprojected_polygon = shapely_transform(transformer.transform, polygon)

                # the images making up the stitched_image were raster memoryfiles but after using the stitching functioin they no longer are
                # it is now just a numpy array and will have loist the geospatial metadata
                # so the image needs to be turned into a raster memoryfile
                
                # create a MemoryFile dataset from stitched_image

                # changed stitched_image to enhanced_image
                meta = {
                    "driver": "GTiff",
                    "dtype": enhanced_image.dtype,
                    "count": enhanced_image.shape[0],  # Number of bands
                    "height": enhanced_image.shape[1],
                    "width": enhanced_image.shape[2],
                    "transform": correct_transform,  # both transform and correct_transform seem to do the same thing here
                    "crs": "EPSG:4326",  # CRS for your data
                }
                
                with MemoryFile() as memfile:
                    with memfile.open(**meta) as dataset:
                        # write each band to the dataset
                        for i in range(enhanced_image.shape[0]):
                            dataset.write(enhanced_image[i], i + 1)
                
                        # mask the raster using the Local Authoiruty polygon
                        masked_image, masked_transform = mask(
                            dataset, shapes=[reprojected_polygon], crop=False)

                ###### Get data for pixel count and sq km
                valid_pixels = np.count_nonzero(masked_image[0])
                sq_km = ((valid_pixels *10) * 10)/1000000


                # for processing inline with the ViT model / Imagenet
                vit_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize((224, 224)),  # resizes for ViT
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # ImageNet normalisation
                ])
                
                #preprocesses image
                def preprocess_raster_image_vit(raster_image):
                    raster_image = np.transpose(raster_image, (1, 2, 0))  # convert to (H, W, C)
                    raster_image = raster_image.astype(np.float32)  # changes float32
                    preprocessed_image = vit_transform(raster_image)  # applies transforms
                    preprocessed_image = preprocessed_image.unsqueeze(0)  # adds batch dim
                    return preprocessed_image.to(device)
                
                def predict_vit(sections):
                    model.eval()  # sets model to evaluation mode
                    predictions = []
                    prediction_count = {}
                
                    for section in sections:
                        processed_section = preprocess_raster_image_vit(section)
                
                        with torch.no_grad():
                            output = model(processed_section)
                            probabilities = torch.nn.functional.softmax(output, dim=1)
                            predicted_class = torch.argmax(probabilities, dim=1).item()
                
                        predictions.append(predicted_class)
                        prediction_count[predicted_class] = prediction_count.get(predicted_class, 0) + 1
                
                    return predictions, prediction_count
                
                # create binary mask
                # generate a binary mask, same shapes as image, where valid areas = 1, masked areas = 0
                # to ensure only areas inside boundary are predicted on
                binary_mask = (masked_image.sum(axis=0) > 0).astype(np.uint8)  # sums across channels to check for nonzero pixels
            
                
                # splits into valid sections, ignoring masked area
                sections, coordinates = split_into_sections(masked_image, binary_mask, section_size=64)
                
                # predicts on sections
                predictions, prediction_count = predict_vit(sections)

                # get pred count
                prediction_count = {cls: predictions.count(cls) for cls in set(predictions)}

                # append data to dataframe

                new_row = pd.DataFrame([{
                    'local_authority': x,
                    'year': y,
                    'total_pixels': valid_pixels,
                    'total_sqkm': sq_km,
                    0: prediction_count.get(0, 0),
                    1: prediction_count.get(1, 0),
                    2: prediction_count.get(2, 0),
                    3: prediction_count.get(3, 0),
                    4: prediction_count.get(4, 0),
                    5: prediction_count.get(5, 0),
                    6: prediction_count.get(6, 0),
                    7: prediction_count.get(7, 0),
                    8: prediction_count.get(8, 0),
                    9: prediction_count.get(9, 0)
                }])

                prediction_df = pd.concat([prediction_df, new_row], ignore_index=True)

                progress_count += 1

                print(f'{progress_count},{x}, {y}, {prediction_count}, done')
            except Exception as e:
                print(f"Failed for {x}, {y}. Error: {e}")

    return prediction_df

In [19]:
%%time
prediction_df = make_predictions(all_gdf,model,template_df)

1,South Ayrshire, 2022, {0: 100, 1: 76, 2: 1161, 3: 132, 4: 53, 5: 1415, 6: 29, 7: 85, 8: 56, 9: 4}, done
2,Huntingdonshire, 2022, {0: 622, 1: 6, 2: 92, 3: 335, 4: 65, 5: 959, 6: 93, 7: 114, 8: 20, 9: 18}, done
3,Tewkesbury, 2022, {0: 212, 1: 56, 2: 236, 3: 33, 4: 16, 5: 493, 6: 36, 7: 6, 8: 4, 9: 1}, done
4,West Lancashire, 2022, {0: 348, 1: 10, 2: 403, 4: 3, 5: 96, 6: 30, 7: 2, 9: 5}, done
5,Darlington, 2022, {0: 198, 1: 22, 2: 83, 3: 4, 4: 9, 5: 188, 6: 8, 7: 8}, done
6,Cheltenham, 2022, {0: 15, 1: 8, 2: 74, 3: 2, 4: 4, 5: 14, 6: 5, 7: 7}, done
7,Cambridge, 2022, {0: 6, 2: 9, 3: 4, 4: 15, 5: 5, 6: 9, 7: 62, 8: 1}, done
8,Erewash, 2022, {0: 8, 2: 19, 3: 19, 4: 13, 5: 161, 6: 5, 7: 63}, done
9,West Berkshire, 2022, {0: 440, 1: 14, 2: 360, 3: 96, 4: 36, 5: 744, 6: 19, 7: 62, 8: 7, 9: 2}, done
10,Teignbridge, 2022, {0: 231, 1: 7, 2: 733, 3: 6, 4: 2, 5: 710, 6: 7, 8: 3, 9: 6}, done
11,Portsmouth, 2022, {0: 1, 2: 12, 3: 4, 4: 15, 6: 1, 7: 36, 8: 2}, done
12,Kingston upon Thames, 2022, {0:

In [20]:
prediction_df

,local_authority,year,total_pixels,total_sqkm,0,1,2,3,4,5,6,7,8,9
0,South Ayrshire,2022,12274610,1227.4610,100,76,1161,132,53,1415,29,85,56,4
1,Huntingdonshire,2022,9137588,913.7588,622,6,92,335,65,959,93,114,20,18
2,Tewkesbury,2022,4052295,405.2295,212,56,236,33,16,493,36,6,4,1
3,West Lancashire,2022,3465954,346.5954,348,10,403,0,3,96,30,2,0,5
4,Darlington,2022,1972276,197.2276,198,22,83,4,9,188,8,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,East Dorset,2022,3461697,346.1697,214,12,84,56,21,411,32,74,9,0
398,Torridge,2022,9688162,968.8162,760,55,396,8,6,1258,4,7,2,8
399,Stockton-on-Tees,2022,1586619,158.6619,44,0,52,35,58,155,7,71,3,0
400,Perth & Kinross,2022,53376489,5337.6489,5664,2905,2839,36,14,1779,58,22,13,33


In [21]:
# save as csv
prediction_df.to_csv('prediction_df_2022_v3.csv',index=False)